In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import tempfile
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from datetime import datetime

pd.set_option("display.max_columns",30)

In [2]:
from station.data_submission import DataSubmission
from station.station import StationData


from station.dat_to_nc_all_var import DatToNcAllVar
from station.dat_to_nc_all_var_copy import DatToNcConverter2

In [7]:
# Instantiate the converter
name = "marshall_hourly"
directory = "measurements/Marshall"
target_directory = "station_data_as_nc"


converter = DatToNcConverter2(
    name=name,
    directory=directory,
    target_directory=target_directory,
    hourly=True,
    save_raw=True, 
    raw_directory="station_data_raw",
    save_processed=True,
    processed_directory="station_data_processed"
)

In [8]:
converter.execute()

Reading .dat files: 100%|██████████| 2465/2465 [00:14<00:00, 168.73it/s]


Resampling data to hourly intervals...


Resampling variables: 100%|██████████| 22/22 [01:11<00:00,  3.24s/it]

Saving to station_data_as_nc/marshall_hourly.nc
NetCDF file saved successfully.
Data processing complete.


In [36]:
ds = xr.open_dataset("station_data_as_nc/marshall_allvar.nc")

In [37]:
ds

<xarray.Dataset> Size: 11MB
Dimensions:      (time: 59151, lat: 1, lon: 1)
Coordinates:
  * time         (time) datetime64[ns] 473kB 2017-03-01T05:00:00 ... 2023-11-...
  * lat          (lat) float64 8B 39.95
  * lon          (lon) float64 8B -105.2
Data variables: (12/22)
    bmp180_temp  (time, lat, lon) float64 473kB ...
    bmp180_pres  (time, lat, lon) float64 473kB ...
    bmp180_slp   (time, lat, lon) float64 473kB ...
    bmp180_alt   (time, lat, lon) float64 473kB ...
    bmp280_temp  (time, lat, lon) float64 473kB ...
    bmp280_pres  (time, lat, lon) float64 473kB ...
    ...           ...
    tipping      (time, lat, lon) float64 473kB ...
    vis_light    (time, lat, lon) float64 473kB ...
    ir_light     (time, lat, lon) float64 473kB ...
    uv_light     (time, lat, lon) float64 473kB ...
    wind_dir     (time, lat, lon) float64 473kB ...
    wind_speed   (time, lat, lon) float64 473kB ...

In [38]:
raw_vienna = converter.raw_df
processed_vienna = converter.processed_df
resampled_vienna = converter.resampled_df

In [39]:
raw_vienna.head()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,bme_slp,bme_alt,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
datetime,,,,,,,,,,,,,,,,,,,,,,
2017-03-01 05:24:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,-999.99
2017-03-01 05:25:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.00
2017-03-01 05:26:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.00
2017-03-01 05:27:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.00
2017-03-01 05:28:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.00


In [32]:
processed_vienna.head()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,bme_slp,bme_alt,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
datetime,,,,,,,,,,,,,,,,,,,,,,
2017-04-24 07:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
2017-04-24 07:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,70.1,0.0
2017-04-24 07:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0
2017-04-24 07:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,76.7,0.0
2017-04-24 07:18:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0


In [33]:
resampled_vienna.head() 

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,bme_slp,bme_alt,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
datetime,,,,,,,,,,,,,,,,,,,,,,
2017-04-24 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,23.349669,NaN
2017-04-24 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,20.898067,NaN
2017-04-24 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,19.371252,NaN
2017-04-24 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,44.898214,0.398644
2017-04-24 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:

def df_statistics(df):
    """
    Computes statistics for a pandas DataFrame with a datetime index.
    
    Parameters:
    df (pd.DataFrame): DataFrame with datetime index and measurement columns.
    
    Returns:
    pd.DataFrame: A summary DataFrame containing:
        - Total Potential Measurements
        - Actual Measurements (Non-NaN)
        - Percentage of Measurements Present
        - Non-Zero Measurements
        - Percentage of Non-Zero Measurements

    Use processed Dataframe where -999.9 values are replaced with NaN and values out of bounds are replaced with NaN
    """
    if not isinstance(df.index, pd.DatetimeIndex):
        try:
            df.index = pd.to_datetime(df.index) # import since index is dtype object before
        except Exception as e:
            raise ValueError(f"Failed to convert index to DatetimeIndex: {e}")
    
    
    df = df.sort_index()
    
    # Create a complete minute-wise index
    full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq="min")  # min stands for minute frequency
    
    # Reindex the dataframe to include all minutes, filling missing with NaN
    df = df.reindex(full_index)    
    
    total_minutes = full_index.shape[0]
    
    # Number of potential measurements per column
    total_potential = total_minutes
    
    # Actual measurements (non-NaN) per column
    actual_measurements = df.notna().sum()
    
    # Percentage of measurements present
    percentage_present = (actual_measurements / total_potential) * 100
    
    # Non-zero measurements per column (assuming numeric data)
    non_zero_measurements = (df != 0).sum()
    
    # Percentage of non-zero measurements
    percentage_non_zero = (non_zero_measurements / total_potential) * 100
    
    # Compile the statistics into a summary DataFrame
    summary = pd.DataFrame({
        'Total Potential Measurements': total_potential,
        'Actual Measurements': actual_measurements,
        'Percentage Present (%)': percentage_present.round(2),
        'Non-Zero Measurements': non_zero_measurements,
        'Percentage Non-Zero (%)': percentage_non_zero.round(2)
    })
    
    return summary

# Example Usage:
# Assuming you have a dataframe `df` with a datetime index and multiple columns.

# import pandas as pd
# df = pd.read_csv('your_data.csv', parse_dates=True, index_col='datetime_column')
# stats = dataframe_statistics(df)
# print(stats)


In [39]:
def dataset_statistics(ds, time_dim='time', freq='h'):
    """
    Computes statistics for an xarray Dataset with a time dimension.
    
    Parameters:
    ds (xr.Dataset): xarray Dataset with a time dimension and multiple data variables.
    time_dim (str): Name of the time dimension. Default is 'time'.
    freq (str): Frequency string for expected measurements (default 'T' for minutes).
    
    Returns:
    pd.DataFrame: A summary DataFrame containing statistics for each data variable:
        - Total Potential Measurements
        - Actual Measurements (Non-NaN)
        - Percentage Present (%)
        - Non-Zero Measurements
        - Percentage Non-Zero (%)
    """
    
    # Step 1: Validate Time Dimension
    if time_dim not in ds.dims:
        raise ValueError(f"The Dataset does not contain a '{time_dim}' dimension.")
    
    # Extract the time coordinates
    time = ds.coords[time_dim]
    
    # Ensure the time coordinate is of datetime type
    if not np.issubdtype(time.dtype, np.datetime64):
        try:
            ds = ds.copy()
            ds[time_dim] = pd.to_datetime(ds[time_dim].values)
            time = ds.coords[time_dim]
        except Exception as e:
            raise ValueError(f"Failed to convert '{time_dim}' to datetime: {e}")
    
    # Sort the Dataset by time
    ds = ds.sortby(time_dim)
    
    # Step 2: Create a Complete Time Index
    start_time = pd.to_datetime(time.values.min())
    end_time = pd.to_datetime(time.values.max())
    full_time = pd.date_range(start=start_time, end=end_time, freq=freq)
    total_potential = len(full_time)
    
    # Reindex the Dataset to include all expected timestamps
    ds = ds.reindex({time_dim: full_time})
    
    # Step 3: Initialize a Dictionary to Hold Statistics
    stats_dict = {
        'Total Potential Measurements': total_potential,
        'Actual Measurements': {},
        'Percentage Present (%)': {},
        'Non-Zero Measurements': {},
        'Percentage Non-Zero (%)': {}
    }
    
    # Step 4: Iterate Over Data Variables and Compute Statistics
    for var in ds.data_vars:
        data = ds[var]
        
        # Ensure the data is numeric; skip non-numeric variables
        if not np.issubdtype(data.dtype, np.number):
            print(f"Skipping non-numeric variable '{var}'.")
            continue
        
        # Actual Measurements: Count of non-NaN values
        actual = data.notnull().sum().item()
        
        # Percentage Present
        perc_present = (actual / total_potential) * 100
        
        # Non-Zero Measurements
        non_zero = (data != 0).sum().item()
        
        # Percentage Non-Zero
        perc_non_zero = (non_zero / total_potential) * 100
        
        # Populate the statistics dictionary
        stats_dict['Actual Measurements'][var] = actual
        stats_dict['Percentage Present (%)'][var] = round(perc_present, 2)
        stats_dict['Non-Zero Measurements'][var] = non_zero
        stats_dict['Percentage Non-Zero (%)'][var] = round(perc_non_zero, 2)
    
    # Step 5: Convert the Dictionary to a pandas DataFrame
    summary_df = pd.DataFrame(stats_dict)
    
    # Reorder the DataFrame for better readability
    summary_df = summary_df[
        ['Total Potential Measurements',
         'Actual Measurements',
         'Percentage Present (%)',
         'Non-Zero Measurements',
         'Percentage Non-Zero (%)']
    ]
    
    return summary_df


In [10]:
def counting_tipping_values(station_data, variabel, raworprocessed):
    # Number of potential measurement
    potential_measurements = station_data.shape[0]

    # Count the number of measurements
    measurement_count = station_data[variabel].count()

    # Count the number of times tipping is 0.0
    tipping_zero_count = station_data[station_data[variabel] == 0.0].shape[0]

    # Count the number of times tipping is non-zero and not NaN
    tipping_non_zero_count = station_data[(station_data[variabel] != 0.0) & (station_data[variabel].notna())].shape[0]

    # Total count of rows where tipping is not NaN
    tipping_total_count = station_data[station_data[variabel].notna()].shape[0]

    # Print the counts
    print("Raw or processed:", raworprocessed)
    print(f"Potential measurements: {potential_measurements}")
    print(f"Tipping equals 0.0: {tipping_zero_count}")
    print(f"Tipping is non-zero and not NaN: {tipping_non_zero_count}")
    print(f"Total number of tipping measurements (non-NaN): {tipping_total_count}")

    # Calculate percentages
    tipping_zero_percentage = (tipping_zero_count / tipping_total_count) * 100
    tipping_non_zero_percentage = (tipping_non_zero_count / tipping_total_count) * 100

    print(f"Percentage of tipping values equal to 0.0: {tipping_zero_percentage:.2f}%")
    print(f"Percentage of tipping values non-zero: {tipping_non_zero_percentage:.2f}%")

# Marshall

## Basic

In [11]:
marshall_raw_df = pd.read_csv("station_data_raw/marshall_hourly_raw_data.csv", index_col=0)
marshall_processed_df = pd.read_csv("station_data_processed/marshall_hourly_processed_data.csv", index_col=0)
ds_marshall = xr.open_dataset("station_data_as_nc/marshall_hourly.nc")

In [38]:
ds_marshall

<xarray.Dataset> Size: 11MB
Dimensions:      (time: 59151, lat: 1, lon: 1)
Coordinates:
  * time         (time) datetime64[ns] 473kB 2017-03-01T05:00:00 ... 2023-11-...
  * lat          (lat) float64 8B 39.95
  * lon          (lon) float64 8B -105.2
Data variables: (12/22)
    bmp180_temp  (time, lat, lon) float64 473kB ...
    bmp180_pres  (time, lat, lon) float64 473kB ...
    bmp180_slp   (time, lat, lon) float64 473kB ...
    bmp180_alt   (time, lat, lon) float64 473kB ...
    bmp280_temp  (time, lat, lon) float64 473kB ...
    bmp280_pres  (time, lat, lon) float64 473kB ...
    ...           ...
    tipping      (time, lat, lon) float64 473kB ...
    vis_light    (time, lat, lon) float64 473kB ...
    ir_light     (time, lat, lon) float64 473kB ...
    uv_light     (time, lat, lon) float64 473kB ...
    wind_dir     (time, lat, lon) float64 473kB ...
    wind_speed   (time, lat, lon) float64 473kB ...

In [12]:
marshall_processed_df.describe()


,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,bme_slp,bme_alt,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
count,2.413617e+06,2.414535e+06,2.413942e+06,2409843.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.612900e+06,2.400251e+06,2.551416e+06,2.759821e+06,1.735939e+06,1.749167e+06,1.736749e+06,2.762346e+06,2.692444e+06
mean,1.118363e+01,8.224099e+02,1.010259e+03,1743.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.151809e+01,4.692207e+01,1.087879e+01,6.877982e-04,6.011301e+02,3.866441e+03,1.872449e+02,1.445796e+02,1.391728e+00
std,1.063827e+01,1.924599e+01,2.409141e+01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.057538e+01,2.364455e+01,1.061440e+01,7.311341e-02,2.881190e+02,2.904967e+03,1.460509e+02,1.208759e+02,1.780924e+00
min,-3.680000e+01,3.069000e+02,3.663300e+02,1743.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.950000e+01,1.000000e-01,-3.470000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.300000e+00,8.210200e+02,1.005980e+03,1743.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.700000e+00,2.820000e+01,3.100000e+00,0.000000e+00,2.630000e+02,2.800000e+02,3.000000e+00,0.000000e+00,0.000000e+00
50%,1.170000e+01,8.251000e+02,1.012180e+03,1743.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.200000e+01,4.350000e+01,1.110000e+01,0.000000e+00,6.530000e+02,4.325000e+03,2.120000e+02,1.494000e+02,8.500000e-01
75%,1.930000e+01,8.285400e+02,1.018980e+03,1743.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.950000e+01,6.480000e+01,1.890000e+01,0.000000e+00,8.650000e+02,6.621000e+03,3.260000e+02,2.507000e+02,2.110000e+00
max,4.500000e+01,1.097700e+03,1.353760e+03,1743.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.490000e+01,9.990000e+01,4.390000e+01,2.880000e+01,1.919000e+03,1.515500e+04,9.660000e+02,3.600000e+02,9.955000e+01


In [13]:
marshall_processed_df.head()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,bme_slp,bme_alt,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
datetime,,,,,,,,,,,,,,,,,,,,,,
2017-03-01 05:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
2017-03-01 05:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0
2017-03-01 05:26:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0
2017-03-01 05:27:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0
2017-03-01 05:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0


In [14]:
marshall_raw_df.describe()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,bme_slp,bme_alt,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
count,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06,3.549033e+06
mean,-3.120907e+02,2.398543e+02,3.673190e+02,8.625385e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-2.552583e+02,-2.919516e+02,-2.732406e+02,-2.223711e+02,-2.168339e+02,1.398466e+03,-4.190067e+02,-1.091284e+02,-2.403002e+02
std,4.716672e+02,8.500179e+02,9.378128e+02,1.280580e+03,5.798030e-12,5.798030e-12,5.798030e-12,5.798030e-12,5.798030e-12,5.798030e-12,5.798030e-12,5.798030e-12,5.798030e-12,4.458262e+02,4.902180e+02,4.544957e+02,4.158371e+02,8.253469e+02,3.174661e+03,6.022131e+02,4.872279e+02,4.285013e+02
min,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02
25%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,0.000000e+00,-9.999900e+02,-9.999900e+02,-9.999900e+02,0.000000e+00,0.000000e+00
50%,3.900000e+00,8.213500e+02,1.006360e+03,1.743000e+03,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,6.200000e+00,2.890000e+01,4.800000e+00,0.000000e+00,-9.999900e+02,-9.999900e+02,-9.999900e+02,5.950000e+01,1.000000e-01
75%,1.590000e+01,8.268600e+02,1.015330e+03,1.743000e+03,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,1.690000e+01,5.350000e+01,1.600000e+01,0.000000e+00,6.070000e+02,4.032000e+03,1.880000e+02,2.278000e+02,1.640000e+00
max,8.450000e+01,1.097700e+03,1.353760e+03,1.743000e+03,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,1.117000e+02,9.990000e+01,1.239000e+02,2.880000e+01,1.919000e+03,1.515500e+04,9.660000e+02,3.600000e+02,9.955000e+01


In [15]:
marshall_raw_df.head()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,bme_slp,bme_alt,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
datetime,,,,,,,,,,,,,,,,,,,,,,
2017-03-01 05:24:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,-999.99
2017-03-01 05:25:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.00
2017-03-01 05:26:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.00
2017-03-01 05:27:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.00
2017-03-01 05:28:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.00


In [16]:
print(marshall_raw_df.info())
print(marshall_processed_df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 3549033 entries, 2017-03-01 05:24:00 to 2023-11-29 19:56:00
Data columns (total 22 columns):
 #   Column       Dtype  
---  ------       -----  
 0   bmp180_temp  float64
 1   bmp180_pres  float64
 2   bmp180_slp   float64
 3   bmp180_alt   float64
 4   bmp280_temp  float64
 5   bmp280_pres  float64
 6   bmp280_slp   float64
 7   bmp280_alt   float64
 8   bme_temp     float64
 9   bme_pres     float64
 10  bme_slp      float64
 11  bme_alt      float64
 12  bme_hum      float64
 13  htu_temp     float64
 14  htu_hum      float64
 15  mcp9808      float64
 16  tipping      float64
 17  vis_light    float64
 18  ir_light     float64
 19  uv_light     float64
 20  wind_dir     float64
 21  wind_speed   float64
dtypes: float64(22)
memory usage: 622.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 3549033 entries, 2017-03-01 05:24:00 to 2023-11-29 19:56:00
Data columns (total 22 columns):
 #   Column       Dtype  
---  ------       -----  

In [21]:
print(type(marshall_processed_df.index))
print(marshall_processed_df.index.dtype)

<class 'pandas.core.indexes.base.Index'>
object


In [24]:
stats = df_statistics(marshall_processed_df)
print(stats)

             Total Potential Measurements  Actual Measurements  \
bmp180_temp                       3549033              2413617   
bmp180_pres                       3549033              2414535   
bmp180_slp                        3549033              2413942   
bmp180_alt                        3549033              2409843   
bmp280_temp                       3549033                    0   
bmp280_pres                       3549033                    0   
bmp280_slp                        3549033                    0   
bmp280_alt                        3549033                    0   
bme_temp                          3549033                    0   
bme_pres                          3549033                    0   
bme_slp                           3549033                    0   
bme_alt                           3549033                    0   
bme_hum                           3549033                    0   
htu_temp                          3549033              2612900   
htu_hum   

In [25]:
stats = dataframe_statistics_with_reindex(marshall_processed_df)
print(stats)

/var/folders/lj/q1ndb2493275c2y0vzyplnk00000gn/T/ipykernel_40147/3468911865.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='T')  # 'T' stands for minute frequency


             Total Potential Measurements  Actual Measurements  \
bmp180_temp                       3549033              2413617   
bmp180_pres                       3549033              2414535   
bmp180_slp                        3549033              2413942   
bmp180_alt                        3549033              2409843   
bmp280_temp                       3549033                    0   
bmp280_pres                       3549033                    0   
bmp280_slp                        3549033                    0   
bmp280_alt                        3549033                    0   
bme_temp                          3549033                    0   
bme_pres                          3549033                    0   
bme_slp                           3549033                    0   
bme_alt                           3549033                    0   
bme_hum                           3549033                    0   
htu_temp                          3549033              2612900   
htu_hum   

In [40]:
summary = dataset_statistics(ds_marshall)
print(summary)

             Total Potential Measurements  Actual Measurements  \
bmp180_temp                         59151                38990   
bmp180_pres                         59151                39811   
bmp180_slp                          59151                39802   
bmp180_alt                          59151                    0   
bmp280_temp                         59151                    0   
bmp280_pres                         59151                    0   
bmp280_slp                          59151                    0   
bmp280_alt                          59151                    0   
bme_temp                            59151                    0   
bme_pres                            59151                    0   
bme_slp                             59151                    0   
bme_alt                             59151                    0   
bme_hum                             59151                    0   
htu_temp                            59151                42839   
htu_hum   

In [ ]:
counting_tipping_values(marshall_processed_df, "processed")

Raw or processed: processed
Tipping equals 0.0: 45248
Tipping is non-zero and not NaN: 1140
Total number of tipping measurements (non-NaN): 46388
Percentage of tipping values equal to 0.0: 97.54%
Percentage of tipping values non-zero: 2.46%


# Vienna

## Basic

In [41]:
vienna_raw_df = pd.read_csv("station_data_raw/vienna_hourly_raw_data.csv", index_col=0)
vienna_processed_df = pd.read_csv("station_data_processed/vienna_hourly_processed_data.csv", index_col=0)
ds_vienna = xr.open_dataset("station_data_as_nc/vienna_hourly.nc")

In [33]:
vienna_processed_df.head()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,bme_slp,bme_alt,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
datetime,,,,,,,,,,,,,,,,,,,,,,
2017-04-24 07:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
2017-04-24 07:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,70.1,0.0
2017-04-24 07:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0
2017-04-24 07:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,76.7,0.0
2017-04-24 07:18:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0


In [30]:
vienna_processed_df.describe()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,bme_slp,bme_alt,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
count,0.0,0.0,0.0,0.0,1.135884e+06,1.135825e+06,1.135801e+06,1132564.0,0.0,0.0,0.0,0.0,0.0,1.821771e+06,1.779866e+06,886720.000000,1.924638e+06,1.153430e+06,1.153436e+06,1.153434e+06,1.923084e+06,991046.000000
mean,NaN,NaN,NaN,NaN,1.462842e+01,9.648641e+02,9.832735e+02,159.0,NaN,NaN,NaN,NaN,NaN,1.241188e+01,6.878195e+01,10.982326,1.245325e-03,3.386394e+02,1.100471e+03,4.638932e+01,1.833463e+02,0.837312
std,NaN,NaN,NaN,NaN,9.280641e+00,1.023951e+02,1.045270e+02,0.0,NaN,NaN,NaN,NaN,NaN,8.565497e+00,1.775864e+01,8.994721,7.158464e-02,1.650865e+02,1.672651e+03,8.505470e+01,1.100088e+02,0.793955
min,NaN,NaN,NaN,NaN,-3.170000e+01,3.822200e+02,3.891800e+02,159.0,NaN,NaN,NaN,NaN,NaN,-3.970000e+01,1.200000e+00,-15.400000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,NaN,NaN,NaN,NaN,6.700000e+00,9.909500e+02,1.009910e+03,159.0,NaN,NaN,NaN,NaN,NaN,5.400000e+00,5.630000e+01,3.600000,0.000000e+00,2.610000e+02,2.530000e+02,2.000000e+00,8.200000e+01,0.240000
50%,NaN,NaN,NaN,NaN,1.456000e+01,9.973900e+02,1.016170e+03,159.0,NaN,NaN,NaN,NaN,NaN,1.180000e+01,7.030000e+01,9.700000,0.000000e+00,2.630000e+02,2.720000e+02,3.000000e+00,1.974000e+02,0.670000
75%,NaN,NaN,NaN,NaN,2.243000e+01,1.002480e+03,1.021620e+03,159.0,NaN,NaN,NaN,NaN,NaN,1.890000e+01,8.280000e+01,18.000000,0.000000e+00,3.400000e+02,1.023000e+03,4.500000e+01,2.807000e+02,1.240000
max,NaN,NaN,NaN,NaN,4.227000e+01,1.087580e+03,1.107610e+03,159.0,NaN,NaN,NaN,NaN,NaN,3.810000e+01,9.990000e+01,41.700000,2.800000e+01,1.630000e+03,1.354100e+04,7.360000e+02,3.600000e+02,98.240000


In [31]:
vienna_raw_df.describe()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,bme_slp,bme_alt,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
count,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06
mean,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-5.688633e+02,-1.651387e+02,-1.573345e+02,-5.089579e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-3.100451e+02,-2.883837e+02,-6.643431e+02,-2.800229e+02,-4.224009e+02,-9.368240e+01,-5.484988e+02,-1.487072e+02,-6.289503e+02
std,4.661161e-12,4.661161e-12,4.661161e-12,4.661161e-12,5.015931e+02,9.735705e+02,9.827422e+02,5.727031e+02,4.661161e-12,4.661161e-12,4.661161e-12,4.661161e-12,4.661161e-12,4.717284e+02,5.043525e+02,4.761447e+02,4.490081e+02,6.718095e+02,1.513093e+03,5.212565e+02,5.397940e+02,4.834010e+02
min,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02
25%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02
50%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,5.800000e+00,5.620000e+01,-9.999900e+02,0.000000e+00,-9.999900e+02,-9.999900e+02,-9.999900e+02,1.128000e+02,-9.999900e+02
75%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,1.163000e+01,9.956200e+02,1.014450e+03,1.590000e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,1.550000e+01,7.630000e+01,3.500000e+00,0.000000e+00,2.620000e+02,2.540000e+02,3.000000e+00,2.565000e+02,3.600000e-01
max,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,8.147000e+01,1.087580e+03,1.107610e+03,1.590000e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,8.140000e+01,9.990000e+01,1.247000e+02,2.800000e+01,1.630000e+03,1.354100e+04,7.360000e+02,3.600000e+02,9.824000e+01


In [32]:
print(vienna_processed_df.info())
print(vienna_raw_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2673207 entries, 2017-04-24 07:14:00 to 2022-05-24 16:40:00
Data columns (total 22 columns):
 #   Column       Dtype  
---  ------       -----  
 0   bmp180_temp  float64
 1   bmp180_pres  float64
 2   bmp180_slp   float64
 3   bmp180_alt   float64
 4   bmp280_temp  float64
 5   bmp280_pres  float64
 6   bmp280_slp   float64
 7   bmp280_alt   float64
 8   bme_temp     float64
 9   bme_pres     float64
 10  bme_slp      float64
 11  bme_alt      float64
 12  bme_hum      float64
 13  htu_temp     float64
 14  htu_hum      float64
 15  mcp9808      float64
 16  tipping      float64
 17  vis_light    float64
 18  ir_light     float64
 19  uv_light     float64
 20  wind_dir     float64
 21  wind_speed   float64
dtypes: float64(22)
memory usage: 469.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 2673207 entries, 2017-04-24 07:14:00 to 2022-05-24 16:40:00
Data columns (total 22 columns):
 #   Column       Dtype  
---  ------       -----  

In [ ]:
counting_tipping_values(vienna_raw_df, "raw")

Raw or processed: raw
Tipping equals 0.0: 1918917
Tipping is non-zero and not NaN: 754290
Total number of tipping measurements (non-NaN): 2673207
Percentage of tipping values equal to 0.0: 71.78%
Percentage of tipping values non-zero: 28.22%


In [ ]:
counting_tipping_values(vienna_processed_df, "processed")

Raw or processed: processed
Tipping equals 0.0: 30236
Tipping is non-zero and not NaN: 1840
Total number of tipping measurements (non-NaN): 32076
Percentage of tipping values equal to 0.0: 94.26%
Percentage of tipping values non-zero: 5.74%


In [35]:
stats = dataframe_statistics_with_reindex(vienna_processed_df)
print(stats)

/var/folders/lj/q1ndb2493275c2y0vzyplnk00000gn/T/ipykernel_40147/340978915.py:14: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='T')  # 'T' stands for minute frequency


             Total Potential Measurements  Actual Measurements  \
bmp180_temp                       2673207                    0   
bmp180_pres                       2673207                    0   
bmp180_slp                        2673207                    0   
bmp180_alt                        2673207                    0   
bmp280_temp                       2673207              1135884   
bmp280_pres                       2673207              1135825   
bmp280_slp                        2673207              1135801   
bmp280_alt                        2673207              1132564   
bme_temp                          2673207                    0   
bme_pres                          2673207                    0   
bme_slp                           2673207                    0   
bme_alt                           2673207                    0   
bme_hum                           2673207                    0   
htu_temp                          2673207              1821771   
htu_hum   

In [42]:
summary = dataset_statistics(ds_vienna)
print(summary)

             Total Potential Measurements  Actual Measurements  \
bmp180_temp                         44554                    0   
bmp180_pres                         44554                    0   
bmp180_slp                          44554                    0   
bmp180_alt                          44554                    0   
bmp280_temp                         44554                17070   
bmp280_pres                         44554                17072   
bmp280_slp                          44554                17072   
bmp280_alt                          44554                    0   
bme_temp                            44554                    0   
bme_pres                            44554                    0   
bme_slp                             44554                    0   
bme_alt                             44554                    0   
bme_hum                             44554                    0   
htu_temp                            44554                27336   
htu_hum   

# Barbados

In [43]:
barbados_raw_df = pd.read_csv("station_data_raw/barbados_hourly_raw_data.csv", index_col=0)
barbados_processed_df = pd.read_csv("station_data_processed/barbados_hourly_processed_data.csv", index_col=0)
ds_barbados = xr.open_dataset("station_data_as_nc/barbados_hourly.nc")


## Basic

In [ ]:
barbados_raw_df.describe()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,...,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
count,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,...,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06
mean,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-2.123054e+02,4.842709e+02,5.069669e+02,-2.188401e+01,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,-1.967659e+02,-4.363836e+02,-2.603876e+02,-4.356546e+01,9.118536e+01,1.256991e+03,-1.646803e+02,-7.625920e+02,-4.196570e+01
std,2.501111e-12,2.501111e-12,2.501111e-12,2.501111e-12,4.332454e+02,8.225395e+02,8.353257e+02,5.379648e+02,2.501111e-12,2.501111e-12,...,2.501111e-12,4.197850e+02,5.426432e+02,4.592763e+02,2.041328e+02,6.374778e+02,2.569836e+03,4.744023e+02,4.254855e+02,2.072575e+02
min,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02
25%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,2.312000e+01,6.486000e+02,6.693200e+02,2.740000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,9.300000e+00,-9.999900e+02,-9.999900e+02,0.000000e+00,2.590000e+02,2.520000e+02,1.000000e+00,-9.999900e+02,1.240000e+00
50%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,2.485000e+01,9.836200e+02,1.014750e+03,2.740000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,2.350000e+01,6.970000e+01,2.420000e+01,0.000000e+00,2.610000e+02,2.540000e+02,2.000000e+00,-9.999900e+02,2.550000e+00
75%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,2.670000e+01,9.854700e+02,1.016690e+03,2.740000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,2.530000e+01,8.750000e+01,2.590000e+01,0.000000e+00,4.300000e+02,1.988000e+03,9.200000e+01,-9.999900e+02,3.650000e+00
max,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,8.441000e+01,1.084970e+03,1.119660e+03,2.740000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,1.243000e+02,9.990000e+01,1.239000e+02,1.260000e+01,1.960000e+03,1.405200e+04,9.840000e+02,0.000000e+00,9.998000e+01


In [ ]:
barbados_raw_df.head()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,...,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
datetime,,,,,,,,,,,,,,,,,,,,,
2020-06-12 00:00:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,...,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.0
2020-06-12 00:01:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,...,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.0
2020-06-12 00:02:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,...,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.0
2020-06-12 00:03:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,...,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.0
2020-06-12 00:04:00,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,-999.99,...,-999.99,-999.99,-999.99,-999.99,0.0,-999.99,-999.99,-999.99,0.0,0.0


In [ ]:
barbados_processed_df.describe()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,...,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed,tas
count,0.0,0.0,0.0,0.0,19275.000000,19304.000000,19304.000000,0.0,0.0,0.0,...,19539.000000,15417.000000,17389.000000,29008.000000,22612.000000,21491.000000,13230.000000,0.0,26956.000000,24780.000000
mean,NaN,NaN,NaN,NaN,26.363526,984.414959,1015.581027,NaN,NaN,NaN,...,22.188743,85.960735,25.575539,0.193285,419.594529,1990.336373,147.866062,NaN,3.065972,297.753908
std,NaN,NaN,NaN,NaN,1.966420,11.590826,11.953170,NaN,NaN,NaN,...,6.201811,8.186707,2.048191,1.341190,228.350229,2400.179046,129.135906,NaN,2.429038,2.330054
min,NaN,NaN,NaN,NaN,-21.155000,648.600000,669.320000,NaN,NaN,NaN,...,1.500000,55.900000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.001000,251.995000
25%,NaN,NaN,NaN,NaN,24.975000,983.830000,1014.975000,NaN,NaN,NaN,...,21.900000,80.150000,24.200000,0.000000,260.000000,253.500000,29.000000,NaN,1.963417,296.561250
50%,NaN,NaN,NaN,NaN,26.160000,985.000000,1016.210000,NaN,NaN,NaN,...,24.400000,86.900000,25.400000,0.000000,266.500000,499.000000,124.500000,NaN,2.895667,297.826667
75%,NaN,NaN,NaN,NaN,27.660000,986.125000,1017.370000,NaN,NaN,NaN,...,26.200000,92.550000,27.100000,0.000000,536.625000,3262.250000,227.000000,NaN,3.824583,299.106667
max,NaN,NaN,NaN,NaN,33.410000,990.780000,1021.940000,NaN,NaN,NaN,...,31.100000,99.800000,32.200000,66.200000,1221.000000,10258.500000,516.500000,NaN,74.617241,303.870000


In [ ]:
barbados_processed_df.head()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,...,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed,tas
datetime,,,,,,,,,,,,,,,,,,,,,
2020-06-08 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-08 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-08 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-08 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(barbados_processed_df.info())
print(barbados_raw_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 30293 entries, 2020-06-08 15:00:00 to 2023-11-21 23:00:00
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   bmp180_temp  0 non-null      float64
 1   bmp180_pres  0 non-null      float64
 2   bmp180_slp   0 non-null      float64
 3   bmp180_alt   0 non-null      float64
 4   bmp280_temp  19275 non-null  float64
 5   bmp280_pres  19304 non-null  float64
 6   bmp280_slp   19304 non-null  float64
 7   bmp280_alt   0 non-null      float64
 8   bme_temp     0 non-null      float64
 9   bme_pres     0 non-null      float64
 10  bme_slp      0 non-null      float64
 11  bme_alt      0 non-null      float64
 12  bme_hum      0 non-null      float64
 13  htu_temp     19539 non-null  float64
 14  htu_hum      15417 non-null  float64
 15  mcp9808      17389 non-null  float64
 16  tipping      29008 non-null  float64
 17  vis_light    22612 non-null  float64
 18  ir_light     21491 

In [ ]:
counting_tipping_values(barbados_raw_df, "raw")

Raw or processed: raw
Tipping equals 0.0: 1721851
Tipping is non-zero and not NaN: 95657
Total number of tipping measurements (non-NaN): 1817508
Percentage of tipping values equal to 0.0: 94.74%
Percentage of tipping values non-zero: 5.26%


In [ ]:
counting_tipping_values(barbados_processed_df, "processed")

Raw or processed: processed
Tipping equals 0.0: 26140
Tipping is non-zero and not NaN: 2868
Total number of tipping measurements (non-NaN): 29008
Percentage of tipping values equal to 0.0: 90.11%
Percentage of tipping values non-zero: 9.89%


In [37]:
stats = dataframe_statistics_with_reindex(barbados_processed_df)
print(stats)

/var/folders/lj/q1ndb2493275c2y0vzyplnk00000gn/T/ipykernel_40147/340978915.py:14: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='T')  # 'T' stands for minute frequency


             Total Potential Measurements  Actual Measurements  \
bmp180_temp                       1817508                    0   
bmp180_pres                       1817508                    0   
bmp180_slp                        1817508                    0   
bmp180_alt                        1817508                    0   
bmp280_temp                       1817508              1395374   
bmp280_pres                       1817508              1395378   
bmp280_slp                        1817508              1395368   
bmp280_alt                        1817508              1395392   
bme_temp                          1817508                    0   
bme_pres                          1817508                    0   
bme_slp                           1817508                    0   
bme_alt                           1817508                    0   
bme_hum                           1817508                    0   
htu_temp                          1817508              1426764   
htu_hum   

In [44]:
summary = dataset_statistics(ds_barbados)
print(summary)

             Total Potential Measurements  Actual Measurements  \
bmp180_temp                         30293                    0   
bmp180_pres                         30293                    0   
bmp180_slp                          30293                    0   
bmp180_alt                          30293                    0   
bmp280_temp                         30293                19275   
bmp280_pres                         30293                19304   
bmp280_slp                          30293                19304   
bmp280_alt                          30293                    0   
bme_temp                            30293                    0   
bme_pres                            30293                    0   
bme_slp                             30293                    0   
bme_alt                             30293                    0   
bme_hum                             30293                    0   
htu_temp                            30293                19539   
htu_hum   